# Loading data

In [1]:
import json

In [2]:
with open('clinais.train.json') as f:
    data = json.load(f)

In [3]:
from tqdm import tqdm

In [4]:
finalresult = []
for key in tqdm(data['annotated_entries'].keys()):
    ident = data['annotated_entries'][key]['note_id']
    res = []
    tags = []
    gold = data['annotated_entries'][key]['boundary_annotation']['gold']
    currentboundary = ''
    for g in gold:
        res.append(g['span'])
        if(g['boundary'] is None):
            tags.append('I-'+currentboundary)
        else:
            currentboundary = g['boundary']
            tags.append('B-'+currentboundary)
    finalresult.append([ident,res,tags])

# finalresult    

100%|██████████| 781/781 [00:00<00:00, 8403.28it/s]


In [5]:
import numpy as np
import itertools
tags = [x[2] for x in finalresult]
tags = np.unique(list(itertools.chain(*tags)))
id2label = {}
label2id = {}
for i,tag in enumerate(tags):
    id2label[i] = tag
    label2id[tag] = i

In [6]:
finalresult = [[x[0],x[1],[label2id[y] for y in x[2]]] for x in finalresult]
#finalresult[0]


In [7]:
with open('clinais.dev.json') as f:
    data = json.load(f)

In [8]:
finalresultdev = []
for key in tqdm(data['annotated_entries'].keys()):
    ident = data['annotated_entries'][key]['note_id']
    res = []
    tags = []
    gold = data['annotated_entries'][key]['boundary_annotation']['gold']
    currentboundary = ''
    for g in gold:
        res.append(g['span'])
        if(g['boundary'] is None):
            tags.append('I-'+currentboundary)
        else:
            currentboundary = g['boundary']
            tags.append('B-'+currentboundary)
    finalresultdev.append([ident,res,tags])

100%|██████████| 127/127 [00:00<00:00, 8075.87it/s]


In [9]:
finalresultdev = [[x[0],x[1],[label2id[y] for y in x[2]]] for x in finalresultdev]


In [10]:
from datasets import Dataset,DatasetDict

In [11]:
import pandas as pd
df = pd.DataFrame(data=finalresult,columns=['id','tokens','tags'])
dataset_train = Dataset.from_pandas(df)

In [12]:
df = pd.DataFrame(data=finalresultdev,columns=['id','tokens','tags'])
dataset_val = Dataset.from_pandas(df)

In [13]:
dataset = DatasetDict(train=dataset_train,val=dataset_val)

# Processing dataset

In [14]:
from transformers import AutoTokenizer

In [15]:
modelCheckpoint = "joheras/bsc-bio-ehr-es-finetuned-clinais-v2"
tokenizer = AutoTokenizer.from_pretrained(modelCheckpoint)

Downloading:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/863k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/521k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [16]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [17]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 781
    })
    val: Dataset({
        features: ['id', 'tokens', 'tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 127
    })
})

In [19]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [20]:
import evaluate

seqeval = evaluate.load("seqeval")

In [21]:
import numpy as np



def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [22]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    modelCheckpoint, num_labels=len(id2label), id2label=id2label, label2id=label2id
)

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the model checkpoint at joheras/bsc-bio-ehr-es-finetuned-clinais-v2 were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at joheras/bsc-bio-ehr-es-finetuned-clinais-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this mo

In [23]:
training_args = TrainingArguments(
    output_dir="clinico-bsc-bio-ehr-es-finetuned-v2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/joheras/.local/lib/python3.10/site-packages/huggingface_hub/repository.py:705: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.11.
  warnings.warn(
Cloning https://huggingface.co/joheras/clinico-bsc-bio-ehr-es-finetuned-v2 into local empty directory.
The following columns in the training set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: tokens, tags, id. If tokens, tags, id are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
/home/joheras/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 781
  Num Epochs = 100
  Instantaneous batch siz

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/grupoa/config/miniconda3/envs/fastai/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.229020,0.006997,0.013728,0.009269,0.571037
2,No log,0.743649,0.153975,0.194298,0.171802,0.795596
3,No log,0.647677,0.240650,0.312566,0.271934,0.820924
4,No log,0.630915,0.324885,0.446674,0.376167,0.826747
5,No log,0.607595,0.343651,0.457233,0.392388,0.828613
6,No log,0.604123,0.344524,0.455121,0.392175,0.832768
7,No log,0.599924,0.324219,0.438226,0.372699,0.837602
8,No log,0.601609,0.355056,0.500528,0.415425,0.835453
9,No log,0.601861,0.362794,0.537487,0.433191,0.852301
10,No log,0.605167,0.334004,0.525871,0.408532,0.846252


The following columns in the evaluation set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: tokens, tags, id. If tokens, tags, id are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 127
  Batch size = 32
/grupoa/config/miniconda3/envs/fastai/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to clinico-bsc-bio-ehr-es-finetuned-v2/checkpoint-25
Configuration saved in clinico-bsc-bio-ehr-es-finetuned-v2/checkpoint-25/config.json
Model weights saved in clinico-bsc-bio-ehr-es-finetuned-v2/checkpoint-25/pytorch_model.bin
tokenizer config file saved in clinico-bsc-bio-ehr-es-finetuned-v2/chec

TrainOutput(global_step=2500, training_loss=0.07820079441070557, metrics={'train_runtime': 1373.3392, 'train_samples_per_second': 56.869, 'train_steps_per_second': 1.82, 'total_flos': 2.04094907123712e+16, 'train_loss': 0.07820079441070557, 'epoch': 100.0})

In [24]:
trainer.push_to_hub()

Saving model checkpoint to clinico-bsc-bio-ehr-es-finetuned-v2
Configuration saved in clinico-bsc-bio-ehr-es-finetuned-v2/config.json
Model weights saved in clinico-bsc-bio-ehr-es-finetuned-v2/pytorch_model.bin
tokenizer config file saved in clinico-bsc-bio-ehr-es-finetuned-v2/tokenizer_config.json
Special tokens file saved in clinico-bsc-bio-ehr-es-finetuned-v2/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/473M [00:00<?, ?B/s]

Upload file runs/May03_08-55-52_minion/events.out.tfevents.1683096960.minion.544219.0:   0%|          | 1.00/5…

To https://huggingface.co/joheras/clinico-bsc-bio-ehr-es-finetuned-v2
   b9dfad0..ac1e249  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.4799685781618225}, {'name': 'Recall', 'type': 'recall', 'value': 0.6451953537486801}, {'name': 'F1', 'type': 'f1', 'value': 0.5504504504504505}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.856625961103573}]}
To https://huggingface.co/joheras/clinico-bsc-bio-ehr-es-finetuned-v2
   ac1e249..7b50a59  main -> main



'https://huggingface.co/joheras/clinico-bsc-bio-ehr-es-finetuned-v2/commit/ac1e2491282b88d1178e458d1197e8d0cbca00a6'

In [25]:
!rm -rf clinico-bsc-bio-ehr-es-finetuned

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
